In [1]:
from fastapi import FastAPI, UploadFile, File, HTTPException
from dotenv import load_dotenv
from pydantic import BaseModel
import os
from langchain_nomic import NomicEmbeddings
from langchain_milvus import Milvus
from langchain_groq import ChatGroq
from upload import upload
from update import update
from search import search
from delete import delete
from store import store

load_dotenv()

app = FastAPI(debug=True)
print("App Initialized")

# Load API keys
NOMIC_API_KEY = os.getenv("NOMIC_API_KEY")
MILVUS_API_KEY = os.getenv("MILVUS_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
MILVUS_URI = os.getenv("MILVUS_URI")

ModuleNotFoundError: No module named 'fastapi'

In [ ]:
# Initialize embeddings
embeddings = NomicEmbeddings(model="nomic-embed-text-v1", dimensionality=768, nomic_api_key=NOMIC_API_KEY)
print("Embeddings Created")

# Initialize vector store
index_params = {
    "metric_type": "IP",
    "index_type": "HNSW",
    "params": {"M": 8, "efConstruction": 10},
}
 
vector_store = Milvus(
    collection_name="resumes11",
    embedding_function=embeddings,
    connection_args={
        "uri": MILVUS_URI,
        "token": MILVUS_API_KEY
    },
    vector_field="content_dense",
    consistency_level="Strong",
    auto_id=True,
    primary_field="id",
    index_params=index_params,
)
print("Milvus collection initialized")
 
# Initialize LLM
llm = ChatGroq(model="llama3-70b-8192", temperature=0.3, GROQ_API_KEY = "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")
print("LLM Initialized")


In [ ]:
@app.post("/store")  # Store in MongoDB and VectorDB
def calling_store(resume_data: ResumeData):
    """Accepts JSON format data and stores it."""
    try:
        store(resume_data.json_data)  # Pass JSON data to store function
        return {"message": "Successfully stored"}
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error storing data: {str(e)}")
